# Proyecto I - Aprendizaje Máquina

## Integrantes:
- **Cristobal Hachim**
- **Ignacio Duarte**
- **Cristopher Arredondo**

## Introducción
Los Sistemas de Grupos Relacionados por el Diagnóstico (GRD) constituyen un método estandarizado para agrupar estancias hospitalarias en categorías clínicas homogéneas, cuyo objetivo es reflejar la complejidad del paciente y el consumo de recursos asociados. Tradicionalmente, la asignación de un GRD se realiza al alta mediante reglas fijas basadas en el diagnóstico principal, las comorbilidades, los procedimientos realizados, la edad y el sexo, lo que limita su capacidad de adaptación a nuevas prácticas clínicas o combinaciones atípicas de diagnósticos. Predecir de forma temprana el GRD —incluso al ingreso o durante las primeras horas de hospitalización— permitiría a los hospitales anticipar necesidades de camas, personal y suministros, optimizar la planificación presupuestaria y detectar posibles errores de codificación antes de la facturación. Sin embargo, esta tarea plantea un desafío de clasificación multiclase de alta dimensionalidad, dado el enorme número de códigos ICD‑9/ICD‑10 y procedimientos, así como las complejas interacciones entre comorbilidades y tratamientos.

---

**Posible Cambio**

---
Por ello, en este proyecto exploraremos desde técnicas clásicas (regresión logística, árboles de decisión) hasta métodos avanzados de aprendizaje automático (Random Forest, XGBoost, redes neuronales recurrentes y basadas en atención), con el fin de diseñar un sistema preciso, escalable y lo suficientemente interpretable para apoyar la toma de decisiones clínicas y administrativas.

---

## metodologia
---
## 1. Descripción del Dataset

El estudio se basa en el conjunto de datos clínicos del Hospital El Pino, que incluye información detallada sobre diagnósticos (Diag01–Diag35), procedimientos (Proced01–Proced30), variables demográficas (Edad, Sexo), y clasificaciones clínicas como GRD, Severidad y Mortalidad. Además, se incorporan tablas maestras que proporcionan descripciones adicionales para las variables categóricas, enriqueciendo así la comprensión y análisis de los datos.

---

## 2. Método para Desarrollar el Proyecto

Para el desarrollo de este proyecto se adoptó la metodología **CRISP-DM (Cross Industry Standard Process for Data Mining)**, ampliamente utilizada en proyectos de minería de datos por su enfoque estructurado y adaptable a múltiples contextos. Esta metodología permite una gestión clara del ciclo de vida de análisis de datos clínicos.

Esta metodología comprende las siguientes fases:

- **Comprensión del negocio**:  Definir los objetivos del proyecto y los requerimientos desde una perspectiva empresarial.
- **Comprensión de los datos**: Recopilar y familiarizarse con los datos disponibles, identificando problemas de calidad y comprendiendo su estructura.
- **Preparación de los datos**: Seleccionar, limpiar y transformar los datos para su análisis.
- **Modelado**: Seleccionar y aplicar técnicas de modelado adecuadas.
- **Evaluación**: Evaluar los modelos generados para asegurar que cumplen con los objetivos del negocio.
- **Despliegue**: Implementar el modelo en el entorno de producción para su uso práctico.

---

## 3. Proceso para Desarrollar el Modelo

El desarrollo del modelo seguirá los siguientes pasos:

### Preprocesamiento de Datos:

- Manejo de valores faltantes y outliers.
- Codificación de variables categóricas mediante `CountVectorizer`, para vectorizar la información de las columnas, haciéndolas compatibles con los modelos a utilizar.
- Normalización o estandarización de variables numéricas según sea necesario.

### Selección de Características:

- Utilización de técnicas como análisis de correlación y métodos de importancia de variables para seleccionar las características más relevantes.

### División del Conjunto de Datos:

- Separación del conjunto de datos en conjuntos de entrenamiento (80%) y prueba (20%).

### Entrenamiento de Modelos:

- Aplicación de los modelos seleccionados (**Regresión Logística, XGBoost, Redes Neuronales Artificiales**) utilizando el conjunto de entrenamiento.

### Evaluación de Modelos:

Los modelos fueron evaluados según:
- Accuracy
- Precision / Recall
- F1-Score
- AUC-ROC multiclase (one-vs-rest)

### Selección del Mejor Modelo:

- Comparación de los modelos basándose en las métricas de evaluación para seleccionar el más adecuado para el problema en cuestión.

---

## 4. Técnicas de Aprendizaje Automático y Justificación

### a. Regresión Logística

La regresión logística es un modelo supervisado que permite estimar la probabilidad de pertenencia a una clase categórica a partir de un conjunto de variables predictoras. Aunque se diseñó originalmente para clasificación binaria, es fácilmente extendible a problemas multiclase, como es el caso de la clasificación de GRDs.

En este proyecto, se busca predecir el GRD asignado al alta hospitalaria, utilizando como variables de entrada los diagnósticos principales y secundarios, procedimientos realizados, edad y sexo del paciente. La regresión logística se adapta bien a este tipo de problema porque:

- Ofrece interpretabilidad clínica: Permite entender cómo cada diagnóstico o procedimiento afecta la probabilidad de que un paciente pertenezca a un GRD determinado. Esto es clave para contextos de salud donde se requiere transparencia en la toma de decisiones.

- Es robusta frente a datos estructurados y puede manejar gran cantidad de variables categóricas tokenizadas.

- Sirve como modelo base o punto de partida para comparar con algoritmos más complejos.

Por ejemplo, puede identificar qué tan probable es que un paciente con una combinación de diagnósticos obstétricos y ciertos procedimientos quirúrgicos sea clasificado en el GRD “146101”, asociado a partos por cesárea.


### b. XGBoost (Extreme Gradient Boosting)

XGBoost es una implementación eficiente y regularizada del algoritmo gradient boosting, especialmente diseñado para alto rendimiento en tareas de clasificación sobre conjuntos de datos estructurados.

- Maneja interacciones no lineales entre múltiples características clínicas (diagnósticos, procedimientos, edad, etc.).

- Es resistente al sobreajuste, gracias a técnicas de regularización y poda de árboles.

- Funciona bien con grandes volúmenes de datos clínicos donde hay muchos valores categóricos convertidos en vectores.

-Puede detectar relaciones complejas entre combinaciones de diagnósticos y GRDs que modelos lineales no capturan.

En el contexto hospitalario, XGBoost puede aprender patrones sutiles en la combinación de variables clínicas que determinan la clasificación GRD, incluso en presencia de clases desbalanceadas o datos con ruido.

### c. Redes Neuronales Artificiales (ANN)

Las Redes Neuronales Artificiales están compuestas por capas de unidades llamadas neuronas, organizadas en una estructura de red que permite modelar relaciones altamente no lineales y jerárquicas en los datos.

- Las ANN son apropiadas cuando el volumen de datos es grande, como ocurre en este caso con más de 60 columnas de entrada (diagnósticos + procedimientos + variables demográficas).
- Son potentes clasificadores multiclase, capaces de generalizar patrones incluso cuando no hay una relación lineal evidente entre las variables y el GRD.

- Tienen la capacidad de detectar combinaciones complejas entre diagnósticos y procedimientos que otros modelos podrían pasar por alto.

- Si se utilizan arquitecturas como el Perceptrón Multicapa (MLP), se puede ajustar la red para obtener equilibrio entre interpretabilidad, precisión y eficiencia.

En un entorno de este proyecto, las redes neuronales pueden predecir GRDs complejos con alta precisión, lo que permitiría optimizar la asignación de recursos hospitalarios como pabellones, procedimiento, etc. según el perfil clínico del paciente.

---

## 5. Métricas para Evaluar la Calidad de los Modelos

Para evaluar y comparar el rendimiento de los modelos, se utilizarán las siguientes métricas:

- **Precisión (Accuracy)**: Proporción de predicciones correctas sobre el total de casos.
- **Precisión (Precision)**: Proporción de verdaderos positivos sobre el total de predicciones positivas.
- **Sensibilidad (Recall)**: Proporción de verdaderos positivos sobre los positivos reales.
- **F1-Score**: Media armónica entre precisión y sensibilidad.
- **Area bajo la curva ROC (AUC-ROC)**: Mide la capacidad del modelo para distinguir entre clases.​


### Justificación de Métricas:
Estas métricas proporcionan una visión integral del rendimiento de los modelos, especialmente en contextos donde las clases pueden estar desbalanceadas. Por ejemplo, en aplicaciones clínicas, es crucial minimizar los falsos negativos, lo que hace que métricas como la sensibilidad y el AUC-ROC sean particularmente relevantes.​



---
